In [31]:
# @title Импорт библиотек

# Фиксируем версии библиотек на 22.11.2023
!pip  install  tiktoken==0.5.1
!pip  install  langchain==0.0.339
!pip  install  openai==1.3.4
!pip  install  faiss-cpu==1.7.4
!pip install gspread==3.4.2
!pip install ipywidgets
# !pip install -q ipywidgets#==8.0.6
!pip install loguru

from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import MarkdownHeaderTextSplitter

import requests
import os
import re
import getpass
import openai
import tiktoken
from openai import OpenAI
import zipfile
from IPython import display
import timeit
import time
from collections import Counter
import ipywidgets as widgets
# from ipywidgets import AppLayout, Button, Layout

import gspread                  # Импортируем API для работы с Google таблицами
from google.colab import auth   # Импортируем модуль для аутентификации
from google.auth import default # Импортируем модуль для работы с учетными данными

from ipywidgets import AppLayout, Button, Layout
from IPython import display

# Очистить экран.
display.clear_output()
# print(widgets.__version__)

In [164]:
# @title Главное окно.    Для обновления окна нужно выбрать эту ячейку и нажать - Ctrl+F10

header_layout = widgets.Layout(width="auto", height="auto", border='1px solid black', padding='3px', margin='3px')
left_layout   = widgets.Layout(width="50", height="auto", border='1px solid black', padding='3px', margin='3px')
right_layout  = widgets.Layout(width="auto", height="auto", border='1px solid black', padding='3px', margin='3px')
center_layout = widgets.Layout(width="auto", height="auto", border='1px solid black', padding='3px', margin='3px')
footer_layout = widgets.Layout(width="auto", height="auto", border='1px solid black', padding='3px', margin='3px')
#left_layout.visibility = 'visible' # ['visible', 'hidden', 'inherit', 'initial', 'unset']


# output_style=dict(
#     font_style='italic',
#     font_weight='bold',
#     font_variant="small-caps",
#     text_color='red',
#     text_decoration='underline'
# )


header = widgets.Output(layout=header_layout)
left = widgets.Output(layout=left_layout)
right = widgets.Output(layout=right_layout)
center = widgets.Output(layout=center_layout)
footer = widgets.Output(layout=footer_layout)

App = widgets.AppLayout(header=None, left_sidebar=left, center=center, right_sidebar=right, footer=None,
        pane_widths=[1, 4, 4],
        # pane_heights=[1, 5, '60px']
)

display.display(App)

AppLayout(children=(Output(layout=Layout(border='1px solid black', grid_area='left-sidebar', height='auto', ma…

# Программа

In [165]:
#@title Вспомогательные функции

def num_tokens_from_string(string: str) -> int:
    """Возвращает количество токенов в строке"""
    # Выбор кодировщика. `cl100k_base`используется для `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002`
    encoding = tiktoken.get_encoding("cl100k_base")
    # Разбивка строки на токены и подсчет из количества.
    num_tokens = len(encoding.encode(string))
    return num_tokens

def split_text(text, verbose=0):
    """ Функция разбивает текст на чанки. """
    # Шаблон MarkdownHeaderTextSplitter по которому будет делится переданный
    # текст в формате Markdown.
    headers_to_split_on = [ ("#",    "Header 1"),
                            ("##",   "Header 2"),
                            ("###",  "Header 3"),
                            ("####", "Header 4")
                        ]
    # Создаем экземпляр спилиттера.
    markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    # Получаем список чанков.
    source_chunks = markdown_splitter.split_text(text)

    # Обработка чанков.
    chank_count = len(source_chunks)
    chank_massage = ""
    for number, chank in enumerate(source_chunks):
        # Добавление информации в метаданные чанка о его номере в базе.
        chank.metadata["chank"]=f'{number+1}/{chank_count}'
        # Вывод количества слов/токенов в фрагменте, если включен режим verbose.
        # if verbose:
        count = num_tokens_from_string(chank.page_content)
        chank_massage += f"\nChank#{number+1}/{chank_count}. Tokens in text = {count}\n{'-' * 20}\n{str(chank)}\n{'=' * 20}"
            # print(f"\n Chank#{number+1}/{chank_count}. Tokens in text = {count}\n {'-' * 20}\n{insert_newlines(str(chank))}\n{'=' * 20}")

    # Возвращение списка фрагментов текста.
    return source_chunks, chank_massage

def create_embedding(data, verbose=0):
    """Функция преобразует текстовую Базу знаний в векторную."""
    # Разбивка текста на чанки.
    source_chunks = []
    source_chunks,chank_massage = split_text(text=data, verbose=verbose)

    # Создание векторной Базы знаний на основе чанков.
    search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(), )
    # Подсчет общего количества токенов во всех чанках.
    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]))
    # Печать сводной информации по созданию векторной Базы знаний.
    # # Стоимость эмбэндинга согласно прайса на 22.11.2023 - 0,0001/1К токенов.
    # # https://openai.com/pricing#language-models
    chank_massage=f"Количество токенов в документе : {count_token}\nЦЕНА запроса: {0.0001*(count_token/1000)} $\n===================="+chank_massage
    return search_index, chank_massage

def load_file(url: str):
    """ Функция загрузки документа по url как текст."""
    try:
        response = requests.get(url) # Получение документа по url.
        response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
        return response.text
    except Exception as e:
        print(e)

def load_search_indexes(url: str, verbose=0):
    """Функция загружает текстовую Базу знаний и преобразует ее в векторную."""
    try:
        return create_embedding(load_file(url), verbose=verbose)
    except Exception as e:
        print(e)

def insert_newlines(text: str, max_len: int = 120) -> str:
    """ Функция форматирует переданный текст по длине
    для лучшего восприятия на экране."""
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def filtred_docs (docs, limit_score):
    """ Функция удаляет из отобранных чанков чанки у которых score выше значения limit_score."""
    if bool(limit_score):
        r = []
        for doc in docs:
            if doc[1] <= limit_score:
                r.append(doc)
    else:
        r = docs
    return r

def answer_index(model, system, topic: str, search_index, temp = 0, seed = 1235, verbose_documents = 0,  verbose_price = 0, top_documents = 3, limit_score = 0.0):
    """ Основная функция которая формирует запрос и получает ответ от OpenAI по заданному вопросу
    на основе векторной Базы знаний. """

    # Выборка релевантных чанков.
    docs = filtred_docs(search_index.similarity_search_with_score(topic, k=top_documents), limit_score)

    message_content = ""            # Контекст для GPT.
    message_content_display = ""    # Контекст для вывода на экран.
    for i, doc in enumerate(docs):
        # Формирование контекста для запроса GPT и показа на экран отобранных чанков.
        message_content = message_content + f'Отрывок документа №{i+1}:{doc[0].page_content}'
        # message_content_display = message_content_display + f"\n Отрывок документа №{i+1}. Chank № {doc[0].metadata.get('chank')}. Score({doc[1]:.4f})\n -----------------------\n{insert_newlines(doc[0].page_content)}\n"
        message_content_display = message_content_display + f"\nОтрывок документа №{i+1}.  Chank № {doc[0].metadata.get('chank')}. Score({doc[1]:.4f})\n{doc[0].page_content}\n-----------------------\n"

    # Отправка запроса к Open AI.
    completion = OpenAI().chat.completions.create(
        model = model[0],
        messages = [
            {"role": "system", "content": system } ,
            {"role": "user", "content": f"Документ с информацией для ответа пользователю : {message_content}.\n\nВопрос клиента: {topic}"}
        ],
        temperature=temp,
        seed = seed,
    )

#======================================================================================
    # Подсчет токенов и стоимости.
    system_fingerprint = completion.system_fingerprint
    prompt_tokens = completion.usage.prompt_tokens
    total_tokens = completion.usage.total_tokens
    price_promt_tokens = prompt_tokens * model[1]/1000
    price_answer_tokens = (total_tokens - prompt_tokens) * model[2]/1000
    price_total_token = price_promt_tokens + price_answer_tokens
    # Вывод на экран стоимости запроса.
    # message_price = f"""\n=======================================================\n{prompt_tokens} токенов использовано на вопрос.
    #             Цена: {price_promt_tokens:.6f} $.\n{total_tokens - prompt_tokens} токенов использовано на ответ.
    #             Цена: {price_answer_tokens:.6f} $.\n{total_tokens} токенов использовано всего. Цена: {price_total_token:.6f} $
    #             \n======================================================= \n"  """
    message_price = f"""=======================================================
    {prompt_tokens:05} - токенов использовано на вопрос.
    {total_tokens - prompt_tokens:05} - токенов использовано на ответ.
    {total_tokens:05} - токенов использовано всего.
    Цена: {price_total_token:.6f} $\n========================================================\n"""
    # Ответ OpenAI.
    return completion.choices[0].message.content, message_content_display, message_price

#======================================================================================
#question_normalization
def question_normalization(text):
    """ Функция нормализует текст вопроса. Удаляет лишние пробелы,
    символы, знаки. Делает первое слово с заглавной буквы."""

    # Удаление символов "!?.,-" из текста
    #text = re.sub(r'[-!?_]', '', text) # '[-!?.,_]'
    # Разделение текста на слова
    words = text.split()
    # # Проход по каждому слову
    # for i in range(len(words)):
    #     # Если слово состоит из заглавных букв и длина больше 1 символа, считаем это аббревиатурой
    #     if words[i].isupper() and len(words[i]) > 1:
    #         continue  # Пропускаем аббревиатуры
    #     else:
    #         words[i] = words[i].lower()  # Преобразуем в нижний регистр
    # # Преобразуем первое слово в строке к верхнему регистру
    #words[0] = words[0].capitalize()
    # Объединяем слова обратно в строку
    return ' '.join(words)
#======================================================================================
def load_bd_text(url: str, verbose=0):
    """ Функция загружает текстовую Базу знаний и
        преобразует ее в векторную."""
    response = requests.get(url) # Получение документа по url.
    response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
    return create_embedding(response.text, verbose=verbose)
#======================================================================================
def load_bd_vect(url: str, verbose=0):
    """ Функция загружает векторную Базу знаний."""
    name_bd = 'federallab_bd_index.zip'
    # Скачивание архива Базы знаний
    response = requests.get(url) # Получение документа по url.
    response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
    # Сохранение архива.
    with open(name_bd, 'wb') as file:
        file.write(response.content)
    # Разархивирование Базы знаний.
    with zipfile.ZipFile(name_bd, 'r') as zip:
        zip.extractall()
    # Загрузка векторной Базы знаний.
    federallab_bd = FAISS.load_local(f'federallab_bd_index', OpenAIEmbeddings())

    # if verbose :
    docs = federallab_bd.similarity_search_with_score('', k=10000)
    docs_sorted = sorted(docs, key=lambda x: int(x[0].metadata.get('chank').split('/')[0]))

    chank_massage = ''
    for doc in docs_sorted:
        count = num_tokens_from_string(doc[0].page_content)
        chank_massage += f"\nChank#{doc[0].metadata.get('chank')}. Tokens in text = {count}\n{'-' * 20}\n{str(doc)}\n{'=' * 20}"

    return federallab_bd, chank_massage
#======================================================================================
def load_bd (url_vect: str, url_text: str, verbose=0):
    """ Функция организует очередность загрузки Базы знаний.
        Сначала идет загрузка векторной базы, если она не загружается,
        то загружается база в текстовом формате и потом преобразуется в векторную."""
    try:
        federallab_bd = load_bd_vect(url_vect, verbose)
        print("Загрузка векторной Базы знаний выполнена успешно.")
        return federallab_bd
    except Exception as e:
        print("По указанной ссылке векторной Базы знаний нет.")
        print(e)
        print("\nИдет загрузка текстовой Базы знаний...")
        try:
            federallab_bd = load_bd_text(url_text, verbose)
            print("\nЗагрузка текстовой Базы знаний выполнена успешно.")
            return federallab_bd
        except Exception as e:
            print("\nПо указанной ссылке текстовой Базы знаний нет.")
            print(e)
            print("\nОшибка загрузки!!")
#======================================================================================
def load_bd_question(url: str):
    """ Функция загружает векторную Базу типовых вопросов."""
    name_bd = 'federallab_bd_question.zip'
    # Скачивание архива Базы знаний
    response = requests.get(url) # Получение документа по url.
    response.raise_for_status()  # Проверка ответа и если была ошибка - формирование исключения.
    # Сохранение архива.
    with open(name_bd, 'wb') as file:
        file.write(response.content)
    # Разархивирование Базы знаний.
    with zipfile.ZipFile(name_bd, 'r') as zip:
        zip.extractall()
    # Загрузка векторной Базы знаний.
    federallab_bd = FAISS.load_local(f'federallab_bd_question', OpenAIEmbeddings())
    docs = federallab_bd.similarity_search_with_score('', k=10000)
    return_massege = f" {len(docs)} шт."
    return federallab_bd, return_massege

def load_bd_question_exel(url: str, input_person_exel: str ):
    """ Функция загружает Базу типовых вопросов из таблицы."""

    creds, _ = default()            # Создаем объект учетных данных на основе аутентификации
    gc = gspread.authorize(creds)   # Создаем клиент для таблиц на основе учетных данных
    # Глобальная переменная список листов.
    worksheet_list = []

    # Название файла с вопросами.
    # Ссылка на общий файл.
    try:
        spreadsheet = gc.open_by_url(url)
        # print(f'Подключились к документу - {spreadsheet.title}')
        # Получаем список всех страниц файла.
        worksheet_list = spreadsheet.worksheets()
        # Устанавливаем задерку в ноль. Нужна если часто обращаемся к гугл таблице.
        time_sleep = 0
        if input_person_exel == 'Все':
            # Названия страниц откуда загружаются вопросы. # Докумов, Макеев, Галина, Петрунин, Шляпников, Бугаев, A.Куцинс
            sheet_persons = "Докумов, Макеев, Галина, Петрунин,  Шляпников, Бугаев, A.Куцинс"
            # Устанавливаем задержку. Иначе может ругаться гугл.
            time_sleep = 30
        else:
            sheet_persons = input_person_exel

        # Список страниц откуда загружаются вопросы.
        list_sheet_persons = sheet_persons.replace(' ', '').split(',')
        # Опреледяем номера страниц откуда загружаем вопросы.
        list_sheet_number = []
        for i, worksheet in enumerate(worksheet_list):
            for person in list_sheet_persons:
                if worksheet.title == person:
                        list_sheet_number.append(i)

        # Номер колонки с вопросами.
        column_question = 1

        # Задаем пустой список вопросов.
        list_question_person = []
        # Выбор всех вопросов.

        for sheet_number in list_sheet_number:
            time.sleep(time_sleep)
            worksheet = worksheet_list[sheet_number]
            # print(f"Загрузка вопросов - {worksheet.title}")
            column = worksheet.col_values(column_question)
            # Заполнение списка запросов информацией с выбранного листа.
            for i in range(len(column)-1):
                row = worksheet.row_values(i+2)
                # Считывание ячеек с контролем их наличия для избежании ошибки чтения.
                question   = row[0] if len(row)>=1 else ""            # Вопрос.
                answer_gpt = row[4] if len(row)>=5 else ""            # Идеальный ответ.

                page_content = question_normalization(question)
                # Словарь запроса.
                metadata = {
                    "line":         i+2,                                # Номер строки в документе. Берем строки с вопросами.
                    "person":       worksheet.title,                    # Автор вопроса.
                    "answer_gpt":   question_normalization(answer_gpt), # Ожидаемый ответ.
                }
                page_content = question_normalization(question)

                if ((not metadata['answer_gpt'] == '') and (not page_content == None)):
                    #print(f" {not metadata['answer'] == ''} and {not page_content == None} = {((not metadata['answer'] == '') and (not page_content == None))} Строка: {metadata['line']}. Ответ:{metadata['answer']}")
                    list_question_person.append(Document(page_content=page_content, metadata=metadata))

        # Проверка на пустой список. Делпем фиктивную строку. Иначе ошибка при создании векторной базы.
        if len(list_question_person) == 0:
            list_question_person.append(Document(page_content='', metadata={"line":0, "person":'', "answer_gpt":''}))
            len_list_question_person = 0
        else:
            len_list_question_person = len(list_question_person)
        # Создание векторной базы вопросов.
        embeddings = OpenAIEmbeddings()
        db_question  = FAISS.from_documents(list_question_person, embeddings)
        # Диапазон строк с вопросами на выбранной странице.
        # print(f'Загрузка списка вопросов завершена. Количество вопросов: {len_list_question_person}.')
        massege_question_count = f"{len_list_question_person} шт."
        return db_question, massege_question_count
    except Exception as e:
        print(e)
        print(f'Ошибка подключения к документу. Проверьте ссылку.')


In [166]:
# @title Формирование оконного интерфейса и логики.
#======================================================================================
# Интерфейс.
#======================================================================================
# Левая боковая панель.
#======================================================================================
"""Блок формирования авторизации на OpenAI."""

width_link   = '250px'
width_button = '110px'

#label_api_key = widgets.Label(value = "API key:")
label_api_key = widgets.HTML(value='<p><b>API key:</b></font></p>')

api_key = widgets.Password(
    value='',
    placeholder='Введите OpenAi API key:',
    layout=Layout(width=width_link, height='28px'),)

login_button = widgets.Button(
    description='Авторизация',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width=width_button, height='28px'),)

def login_OpenAI():
    try:
        if api_key.value:
            login_button.button_style = 'info'
            login_button.description = "..."
            openai.api_key = api_key.value
            os.environ["OPENAI_API_KEY"] = openai.api_key
            completion = OpenAI().chat.completions.create(
                model = 'gpt-3.5-turbo-1106',
                messages = [ {"role": "system", "content": "" }, {"role": "user", "content": ""}]
            )
            label_api_key.value = "API key:   Авторизованы."
            login_button.button_style = 'success'
            login_button.description = "OK!"
        else:
            label_api_key.value = "API key:   ВВЕДИТЕ КЛЮЧ!"
            login_button.button_style = 'warning'
            login_button.button_style = ''
            login_button.description = "Авторизация"
    except Exception as e:
            label_api_key.value = "API key:   НЕВЕРНЫЙ КЛЮЧ!"
            login_button.button_style = 'warning'
            login_button.button_style = ''
            login_button.description = "Авторизация"
    finally:
            button_sent_question_enable()

def on_submit_api_key(api_key): login_OpenAI()

def on_click_login(login_button): login_OpenAI()

api_key.on_submit(on_submit_api_key)

login_button.on_click(on_click_login)

#========================================================================
"""Блок загрузки типовых вопросов в векторной форме."""

label_db_question = widgets.Label(value = "Типовые вопросы векторные:")

link_bd_question = widgets.Text(
    value='https://github.com/terrainternship/GPT_labsud/raw/main/federallab_bd_question.zip',
    layout=Layout(width=width_link, height='28px'),)

button_load_db_question = widgets.Button(
    description='Загрузить',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width=width_button, height='28px'),)

def on_button_clicked_load_db_question(button_load):
    try:
        if not link_bd_question.value==None:
            input_question_client.value = ''
            button_load_db_question.description = 'Загрузка...'
            button_load_db_question.button_style = 'info'
            if button_load_db_question_exel.description=='OK!':
                button_load_db_question_exel.description = 'Загрузить'
                button_load_db_question_exel.button_style = ''
                label_db_question_exel.value = "Типовые вопросы тексовые:"
            global federallab_db_question
            federallab_db_question, return_masseges = load_bd_question(link_bd_question.value)
            label_db_question.value = f"Типовые вопросы векторные: {return_masseges}"
            button_load_db_question.button_style = 'success'
            button_load_db_question.description = "OK!"
        else:
            label_db_question.value = f"Типовые вопросы векторные:   ВВЕДИТЕ ССЫЛКУ!"
            button_load_db_question.button_style = 'warning'
            button_load_db_question.description = 'Загрузить'
            button_load_db_question.button_style = ''
    except Exception as e:
        label_db_question.value = f"Типовые вопросы векторные:   ПРОВЕРЬТЕ ССЫЛКУ!"
        button_load_db_question.button_style = 'warning'
        button_load_db_question.description = 'Загрузить'
        button_load_db_question.button_style = ''
    finally:
            button_sent_question_enable()

button_load_db_question.on_click(on_button_clicked_load_db_question)
#========================================================================

"""Блок загрузки типовых вопросов из таблицы."""

label_db_question_exel = widgets.Label(value = "Типовые вопросы текстовые:")

link_bd_question_exel = widgets.Text(
    value='https://docs.google.com/spreadsheets/d/1O-mbXHfLsFzK4AfkE6yAGtfcB4PisT67dZfAue2oOcQ/edit#gid=1746296964',
    layout=Layout(width='150px', height='28px'),)

button_load_db_question_exel = widgets.Button(
    description='Загрузить',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width=width_button, height='28px'),)

input_person_exel = widgets.Dropdown(
    options=['Все', 'Докумов', 'Макеев', 'Галина', 'Петрунин',  'Шляпников', 'Бугаев', 'A.Куцинс'],
    value='Петрунин',
    description='',
    layout=Layout(width='100px'),)

def on_button_clicked_load_db_question_exel(button_load):
    try:
        auth.authenticate_user()        # Аутентифицируем текущего пользователя Colab
        try:
            if not link_bd_question.value==None:
                input_question_client.value = ''
                button_load_db_question_exel.description = 'Загрузка...'
                button_load_db_question_exel.button_style = 'info'
                if button_load_db_question.description=='OK!':
                    button_load_db_question.description = 'Загрузить'
                    button_load_db_question.button_style = ''
                    label_db_question.value = "Типовые вопросы векторные:"
                global federallab_db_question
                federallab_db_question, return_masseges = load_bd_question_exel(link_bd_question_exel.value, input_person_exel.value)
                label_db_question_exel.value = f"Типовые вопросы текстовые: {return_masseges}"
                button_load_db_question_exel.button_style = 'success'
                button_load_db_question_exel.description = "OK!"
            else:
                label_db_question_exel.value = f"Типовые вопросы текстовые:   ВВЕДИТЕ ССЫЛКУ!"
                button_load_db_question_exel.button_style = 'warning'
                button_load_db_question_exel.description = 'Загрузить'
                button_load_db_question_exel.button_style = ''
        except Exception as e:
            label_db_question_exel.value = f"Типовые вопросы текстовые:   ПРОВЕРЬТЕ ССЫЛКУ!"
            button_load_db_question_exel.button_style = 'warning'
            button_load_db_question_exel.description = 'Загрузить'
            button_load_db_question_exel.button_style = ''
    except Exception as e:
        print(" Ошибка подключения.")
    finally:
            button_sent_question_enable()

def on_change_value_input_person_exel(change):
    on_button_clicked_load_db_question_exel(change)

button_load_db_question_exel.on_click(on_button_clicked_load_db_question_exel)
input_person_exel.observe(on_change_value_input_person_exel, names='value')
#========================================================================

label_db_text   = widgets.Label(value = "База знаний текстовая:")
label_db_vector = widgets.Label(value = "База знаний векторная:")

link_bd_text = widgets.Text(
    value='https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Datadase/LabSudDB_v1.md',
    layout=Layout(width=width_link, height='28px'),)

link_bd_vector = widgets.Text(
    value='https://github.com/terrainternship/GPT_labsud/raw/main/federallab_bd_index_v2.zip',
    layout=Layout(width=width_link, height='28px'),)

button_load_db_text = widgets.Button(
    description='Загрузить',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width=width_button, height='28px'),)

button_load_db_vector = widgets.Button(
    description='Загрузить',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width=width_button, height='28px'),)

federallab_db_string = ''

def on_button_clicked_load_db_text(button_load):
    try:
        button_load_db_text.button_style = 'info'
        button_load_db_text.description = 'Загрузка...'
        if button_load_db_vector.description=='OK!':
            button_load_db_vector.description = 'Загрузить'
            button_load_db_vector.button_style = ''
            label_db_vector.value = "База знаний векторная:"
        global federallab_db, federallab_db_string
        federallab_db, federallab_db_string = load_bd_text(link_bd_text.value)
        button_load_db_text.button_style = 'success'
        button_load_db_text.description = 'OK!'
        label_db_text.value = "База знаний текстовая: Загружена."
    except Exception as e:
        button_load_db_text.button_style = 'warning'
        button_load_db_text.description = 'Загрузить'
        label_db_text.value = "База знаний текстовая: ПРОВЕРТЕ ССЫЛКУ!."
    finally:
            button_sent_question_enable()

def on_button_clicked_load_db_vector(button_load):
    try:
        button_load_db_vector.button_style = 'info'
        button_load_db_vector.description = 'Загрузка...'
        if button_load_db_text.description=='OK!':
            button_load_db_text.description = 'Загрузить'
            button_load_db_text.button_style = ''
            label_db_text.value = "База знаний текстовая:"
        global federallab_db, federallab_db_string
        federallab_db, federallab_db_string  = load_bd_vect(link_bd_vector.value)
        button_load_db_vector.button_style = 'success'
        button_load_db_vector.description = 'OK!'
        label_db_vector.value = "База знаний векторная: Загружена."
    except Exception as e:
        button_load_db_vector.button_style = 'warning'
        button_load_db_vector.description = 'Загрузить'
        label_db_vector.value = "База знаний векторная: ПРОВЕРТЕ ССЫЛКУ!."
    finally:
            button_sent_question_enable()

button_load_db_text.on_click(on_button_clicked_load_db_text)
button_load_db_vector.on_click(on_button_clicked_load_db_vector)

button_load_all = widgets.Button(
    description='ВСЕ!',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width=width_button, height='28px', padding='0px 0px 0px 0px'),)
    # layout=Layout(width='130px', height='28px'),)


def on_button_clicked__load_all(button_load):
    on_button_clicked_load_db_question (button_load)
    on_button_clicked_load_db_vector(button_load)
    on_button_clicked_load_prom(button_load)
    button_sent_question_enable()

button_load_all.on_click(on_button_clicked__load_all)

#========================================================================
label_load_data   = widgets.HTML(value='<p><b>Данные для загрузки</b></font></p>',                 layout=Layout(width='90%', height='35px', padding='10px 0px 0px 0px'))
label_param_name  = widgets.HTML(value = '<p><b>Параметры нейро-консультанта</b></font></p>',      layout=Layout(width='90%', height='35px', padding='5px 0px 0px 0px'))
label_param_cache = widgets.HTML(value = '<p><b>Параметры отбора вопросов из кэша</b></font></p>', layout=Layout(width='90%', height='30px', padding='0px 0px 0px 0px'))
#========================================================================

label_promt = widgets.Label(value = "Инструкция нейро-консультанта:")

link_promt = widgets.Text(
    value='https://raw.githubusercontent.com/terrainternship/GPT_labsud/main/Galina/FLSE_promt',
    layout=Layout(width=width_link, height='28px'))

button_load_promt = widgets.Button(
    description='Загрузить',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width=width_button, height='28px'))
#========================================================================

promt_federallab = ''

def on_button_clicked_load_prom(button_load):
    try:
        button_load_promt.description = 'Загрузка...'
        button_load_promt.button_style = 'info'
        global promt_federallab
        promt_federallab = load_file(link_promt.value)
        button_load_promt.description = 'OK!'
        button_load_promt.button_style = 'success'
        label_promt.value = "Инструкция нейро-консультанта: Загружена."
    except Exception as e:
        button_load_promt.description = 'Загрузка'
        button_load_promt.button_style = 'warning'
        label_promt.value = "Инструкция нейро-консультанта: ПРОВЕРЬТЕ ССЫЛКУ!"

button_load_promt.on_click(on_button_clicked_load_prom)
#========================================================================
# Псевдоним = ['Имя модели', 'Цена токена - вопроса', 'Цена токена - ответа'].
MODEL_GPT_4_1106_PREVIEW = ['gpt-4-1106-preview', 0.01, 0.03]   # 128K tokens
MODEL_GPT_3_5_TURBO_1106 = ['gpt-3.5-turbo-1106', 0.001, 0.002] #  16K tokens

SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_1106

model_gpt = widgets.Dropdown(
    options=[('gpt-3.5-turbo-1106', 1), ('gpt-4-1106-preview', 2)],
    value=1,
    description='Модель GPT OpenAI:     ',
    style = {'description_width': 'initial'},
    layout=Layout(width='99%', height='28px'))

def on_change_value_model(change):
    global SELECT_MODEL_GPT
    if change['new'] == 1: SELECT_MODEL_GPT = MODEL_GPT_3_5_TURBO_1106
    if change['new'] == 2: SELECT_MODEL_GPT = MODEL_GPT_4_1106_PREVIEW

model_gpt.observe(on_change_value_model, names='value')
#========================================================================
temperature = 0.0

temperature_gpt = widgets.FloatSlider(
    value=temperature,
    min=0,
    max=2.0,
    step=0.05,
    description='Вариативность ответа:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style = {'description_width': 'initial'},
    layout=Layout(width='99%', height='28px'),)

def on_change_value_temperature(change):
    global temperature
    temperature = change['new']

temperature_gpt.observe(on_change_value_temperature, names='value')

#========================================================================
chank_threshold = 0.5

input_chank_threshold = widgets.FloatSlider(
    value=chank_threshold, min=0, max=1.0, step=0.01,
    description='Порог отбора чанков:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,  readout_format='.2f',
    style = {'description_width': 'initial'},
    layout=Layout(width='99%', height='28px'),)

def on_change_value_chank_threshold(change):
    global chank_threshold
    chank_threshold = change['new']

input_chank_threshold.observe(on_change_value_chank_threshold, names='value')

#========================================================================
seed = 1235

seed_gpt = widgets.IntText(
    value=seed,
    description='Повторяемость ответа:',
    style = {'description_width': 'initial'},
    layout=Layout(width='210px', height='28px'),)

def on_change_value_seed(change): seed = change['new']

seed_gpt.observe(on_change_value_seed, names='value')

#========================================================================
top_documents = 3

input_top_documents = widgets.IntText(
    value = top_documents,
    description = 'Количество чанков:',
    style = {'description_width': 'initial'},
    layout = Layout(width='170px', height='28px'),)

def on_change_value_top_documents(change):
     global top_documents
     top_documents = change['new']

input_top_documents.observe(on_change_value_top_documents, names='value')

#========================================================================
question_norma = True

input_question_norma = widgets.Checkbox(
    value = question_norma,
    description = 'Нормализация вопроса',
    style = {'description_width': 'initial'},
    layout=Layout(width='90%', height='28px'),)

def on_change_value_question_norma(change):
    global question_norma
    question_norma = change['new']

input_question_norma.observe(on_change_value_question_norma, names='value')
#========================================================================
clear_dialogue_before_new_answer = False

input_clear_dialogue_before_new_answer = widgets.Checkbox(
    value = clear_dialogue_before_new_answer,
    description = 'Режим одиночного вопроса',
    style = {'description_width': 'initial'},
    layout=Layout(width='90%', height='28px'),)

def on_change_value_clear_dialogue_before_new_answer(change):
    global clear_dialogue_before_new_answer
    clear_dialogue_before_new_answer = change['new']

input_clear_dialogue_before_new_answer.observe(on_change_value_clear_dialogue_before_new_answer, names='value')


#========================================================================

question_dialog = False

input_question_dialog = widgets.Checkbox(
    value = question_dialog,
    description = 'Использовать диалог',
    style = {'description_width': 'initial'},
    layout=Layout(width='90%', height='28px'),)

def on_change_value_question_dialog(change):
    global question_dialog
    question_dialog = change['new']
    conversational_memory.clear()

input_question_dialog.observe(on_change_value_question_dialog, names='value')
#========================================================================

question_dialog_window = 2

input_question_dialog_window = widgets.IntText(
    value = question_dialog_window,
    description = 'Глубина диалога:',
    style = {'description_width': 'initial'},
    layout = Layout(width='170px', height='28px'),)

def on_change_value_question_dialog_window(change):
     global question_dialog_window
     question_dialog_window = change['new']

input_question_dialog_window.observe(on_change_value_question_dialog_window, names='value')
#========================================================================

verbose_dialog = False

input_verbose_dialog = widgets.Checkbox(
    value = verbose_dialog,
    description = 'Подробности диалога',
    style = {'description_width': 'initial'},
    layout=Layout(width='90%', height='28px'),)

def on_change_value_verbose_dialog(change):
    global verbose_dialog
    verbose_dialog = change['new']

input_verbose_dialog.observe(on_change_value_verbose_dialog, names='value')
#========================================================================

button_dialog_new = widgets.Button(
    description='Новый диалог',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width='200px', height='28px'),)

def on_button_clicked_dialog_new(button_load):
    conversational_memory.clear()
    output_dialogs.value = ''

button_dialog_new.on_click(on_button_clicked_dialog_new)
#========================================================================

verbose_chank = True

input_verbose_chank = widgets.Checkbox(
    value = verbose_chank,
    description = 'Показать чанки',
    style = {'description_width': 'initial'},
    layout=Layout(width='90%', height='28px'),)

def on_change_value_verbose_chank(change):
    global verbose_chank
    verbose_chank = change['new']

input_verbose_chank.observe(on_change_value_verbose_chank, names='value')

#========================================================================
verbose_price = True

input_verbose_price = widgets.Checkbox(
    value = verbose_price,
    description = 'Показать стоимость ответа',
    style = {'description_width': 'initial'},
    layout=Layout(width='90%', height='28px'),)

def on_change_value_verbose_price(change):
    global verbose_price
    verbose_price = change['new']

input_verbose_price.observe(on_change_value_verbose_price, names='value')

#========================================================================
cache_number = 15

input_cache_number = widgets.IntSlider(
    value=cache_number, min=1, max=20, step=1,
    description='Максимальное число вопросов:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    # readout_format='2f',
    style = {'description_width': 'initial'},
    layout=Layout(width='99%', height='28px'),)

def on_change_value_cache_number(change):
    print(change['new'])
    global cache_number
    cache_number = change['new']
    change_value_question_client()

input_cache_number.observe(on_change_value_cache_number, names='value')

#========================================================================
cache_threshold = 0.5

input_cache_threshold = widgets.FloatSlider(
    value=cache_threshold, min=0, max=1.0, step=0.01,
    description='Порог отбора вопросов:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,  readout_format='.2f',
    style = {'description_width': 'initial'},
    layout=Layout(width='99%', height='28px'),)

def on_change_value_cache_threshold(change):
    global cache_threshold
    cache_threshold = change['new']
    change_value_question_client()

input_cache_threshold.observe(on_change_value_cache_threshold, names='value')

#========================================================================
#========================================================================

output_dialogs = widgets.Textarea(
    value='',
    layout=Layout(width='99.5%', height='500px'),
    continuous_update = True,
)

#========================================================================

input_qestion_from_cache = widgets.Select(
    options=[('        Список вопросов из кэша.', 0)],
    value=0,
    rows=cache_number+1,
    layout=Layout(width='99.5%', height='auto'),
)
#===================================================================================================

#===================================================================================================

def on_change_value_qestion_from_cache(change):
    if not not change['new']:
        # Вопрос/ответ из кэша.
        human = docs[change['new']-1][0].page_content
        ai    = docs[change['new']-1][0].metadata['answer_gpt']
        # Отобразить диалог в окне диалога.
        output_dialogs.value = f"Клиент:\n    {human} \n----------\nМенеджер (кэш):\n    {ai}\n==========\n\n{output_dialogs.value}"
        # Перенести вопрос в строку вопросов.
        # input_question_client.value = human
        # Если используется диалог, то записать вопрос/ответ в память. Иначе
        if question_dialog:
            conversational_memory.save_context({"Human": human}, {"AI": ai})
    # Стереть данные о чанках и стоимости в информационном окне.
    global gpt_chank, gpt_price
    gpt_chank, gpt_price = '',''
    output_textarea_info(promt_federallab, gpt_chank, gpt_price, federallab_db_string)

input_qestion_from_cache.observe(on_change_value_qestion_from_cache, names='value')
#========================================================================

input_question_client = widgets.Text(
    value='',
    placeholder='Введите вопрос.',
    layout=Layout(width='99.5%', height='28px'),
    # description='String:',
    disabled=False,)

button_sent_question = widgets.Button(
    description='>',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width='40px', height='28px'),
)

def button_sent_question_enable(verbose=False):
    if not login_button.description == 'OK!':
        button_sent_question.button_style = ''
        if verbose: print('Введите ключ API OpenAI!')
        return False
    if not button_load_db_vector.description == 'OK!' and not button_load_db_text.description == 'OK!' :
        button_sent_question.button_style = ''
        if verbose: print('Загрузите Базу знаний!')
        return False
    if not button_load_promt.description == 'OK!':
        button_sent_question.button_style = ''
        if verbose: print('Загрузите промт!')
        return False
    if input_question_client.value != '':
        button_sent_question.button_style = 'success'
        return True
    else:
        button_sent_question.button_style = ''
        return False

#===================================================================================================
gpt_answer = ''
gpt_chank = ''
gpt_price = ''

def sent_question(model, system, topic: str, search_index, temp = 0, seed = 1235, verbose_documents = 0,  verbose_price = 0, top_documents = 3, limit_score = 0.0):

    global gpt_answer, gpt_chank, gpt_price
    # Запрос GPT
    gpt_answer, gpt_chank, gpt_price = answer_index(
        model = model,
        system = system,
        topic = topic,
        search_index = federallab_db,
        temp = temp,
        seed = seed,
        verbose_documents = verbose_chank,
        verbose_price = verbose_price,
        top_documents = top_documents,
        limit_score = chank_threshold,)

    return gpt_answer
#===================================================================================================

def change_value_question_client():
    input_qestion_from_cache.options = []
    qestion_from_cache = [('        Список вопросов из кэша.', 0)]
    if input_question_client.value:
        global docs
        docs = filtred_docs(federallab_db_question.similarity_search_with_score(input_question_client.value, k=cache_number), cache_threshold)
        for i, doc in enumerate(docs):
            if not doc[0].page_content == None:
                qestion_from_cache.append((f" #{i+1:02}. (Score: {doc[1]:.4f}).  Вопрос: {doc[0].page_content} ", i+1))
    input_qestion_from_cache.options = qestion_from_cache


def on_change_value_question_client(change):
    if button_sent_question_enable():
        change_value_question_client()

#===================================================================================================
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain, LLMChain, StuffDocumentsChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import SystemMessage
from langchain.callbacks import get_openai_callback

from langchain.callbacks import FileCallbackHandler
from loguru import logger



conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history', return_messages=True, k = question_dialog_window)

class MyCustomHandler_end_1(BaseCallbackHandler):
    def on_llm_end(self, response, **kwargs) -> None:
        # print(f"REPONSE_1: ", response)
        pass

class MyCustomHandler_end_2(BaseCallbackHandler):
    def on_llm_end(self, response, **kwargs) -> None:
        # print(f"REPONSE_2: ", response)
        pass

def conversation_chain(model, topic, temp, federallab_db, top_documents, promt_federallab, verbose_dialog):

    condense_question_template =     """Given the following conversation and a follow up question,
        rephrase the follow up question to be a standalone question, in its original language.

        Chat History:
        {chat_history}
        Follow Up Input: {question}
        Standalone question:"""

    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_question_template)

    # And finally, we do the part that returns the answers
    answer_template = promt_federallab + '\n' + """Answer the question based only on the following context in the original language:
                        {context}

                        Question: {question}
    """
    ANSWER_PROMPT = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template(answer_template)])

    llm = ChatOpenAI(model = model[0], temperature=temperature,)

    retriever = federallab_db.as_retriever(search_kwargs={"k": top_documents})





    if os.path.exists('output.log'): os.remove('output.log')
    logfile = "output.log"
    logger.add(logfile, colorize=False, enqueue=True)
    handler = FileCallbackHandler(logfile)

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm = ChatOpenAI(model = model[0], temperature=temperature, callbacks = [MyCustomHandler_end_1()]),
        retriever = retriever,
        chain_type = "stuff",
        condense_question_prompt = CONDENSE_QUESTION_PROMPT,
        condense_question_llm = ChatOpenAI(model = model[0], temperature=temperature, callbacks = [MyCustomHandler_end_2()]),
        combine_docs_chain_kwargs = {'prompt': ANSWER_PROMPT},
        memory = conversational_memory,
        callbacks = [handler],
        verbose = False,
    )

    with get_openai_callback() as cb:
        answer = conversation_chain.run(topic)

    total_tokens = cb.total_tokens
    prompt_tokens = cb.prompt_tokens
    total_cost = cb.total_cost
    completion_tokens = cb.completion_tokens

    assert total_tokens > 0

    gpt_price_ = f"""=======================================================
    {cb.prompt_tokens:05} - токенов использовано на вопрос.
    {cb.completion_tokens:05} - токенов использовано на ответ.
    {cb.total_tokens:05} - токенов использовано всего.
    Цена: {cb.total_cost:.6f} $\n========================================================"""

    gpt_chank_ = ''
    with open('output.log', 'r') as file:
        gpt_chank_ = file.read()

    return answer, gpt_chank_, gpt_price_


def on_click_button_sent_question(change):

    if not button_sent_question_enable(True):
        return

    button_sent_question.button_style = 'danger'
    button_sent_question.description = '..'

    question = input_question_client.value

    start_group = timeit.default_timer()
    if question_dialog:
        global gpt_chank, gpt_price
        gpt_chank, gpt_price = "",""
        gpt_answer, gpt_chank, gpt_price = conversation_chain(
            model = SELECT_MODEL_GPT,
            topic = question,
            temp = temperature,
            federallab_db = federallab_db,
            top_documents = top_documents,
            promt_federallab = promt_federallab,
            verbose_dialog = verbose_dialog,)
    else:
        gpt_answer = sent_question(
            model = SELECT_MODEL_GPT,
            system = promt_federallab,
            topic = question,
            search_index = federallab_db,
            temp = temperature,
            seed = seed,
            verbose_documents = verbose_chank,
            verbose_price = verbose_price,
            top_documents = top_documents,
            limit_score = chank_threshold,)
    stop_group = timeit.default_timer()



    # Выделение из лога колбэка сформированного вопроса через его поиск.
    # Можно этот вопрос получить и колбэка MyCustomHandler_end_2.
    match = re.search(r'Question: (.*?)\?', gpt_chank)
    new_question = input_question_client.value
    if match: new_question = match.group(1)

    if not verbose_dialog:
        gpt_chank = ''
        gpt_price = ''

    if clear_dialogue_before_new_answer: output_dialogs.value = ''

    if question_dialog:
        output_dialogs.value = f"Клиент(диалог):\n    {input_question_client.value} (LLM: {new_question}?)\n----------\nМенеджер ({stop_group-start_group:.2f} сек.):\n    {gpt_answer}\n==========\n\n{output_dialogs.value}"
    else:
        output_dialogs.value = f"Клиент:\n    {input_question_client.value}\n----------\nМенеджер ({stop_group-start_group:.2f} сек.):\n    {gpt_answer}\n==========\n\n{output_dialogs.value}"

    output_textarea_info(promt_federallab, gpt_chank, gpt_price, federallab_db_string)

    button_sent_question.button_style = 'success'
    button_sent_question.description = '>'
    input_question_client.value = ''

input_question_client.observe(on_change_value_question_client, names='value')
input_question_client.on_submit(on_click_button_sent_question)
button_sent_question.on_click(on_click_button_sent_question)
#===================================================================================================
#===================================================================================================
button_info = widgets.ToggleButtons(
    options=['Чанки', 'Все чанки', 'Промт'],
    value = 'Чанки',
    # description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    # tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    icons=['check'] * 3
)

def output_textarea_info(promt_federallab, gpt_chank, gpt_price, federallab_db_string ):
    page = button_info.value
    Textarea_info.value = ''
    if page == 'Промт':
        Textarea_info.value = promt_federallab
    elif page == 'Чанки':
       Textarea_info.value = gpt_price + gpt_chank
    elif page == 'Все чанки':
        Textarea_info.value = federallab_db_string

def on_change_value_button_info(change):
    output_textarea_info(promt_federallab, gpt_chank, gpt_price, federallab_db_string)


button_info.observe(on_change_value_button_info, names='value')

Textarea_info   = widgets.Textarea(
    value='',
    layout=Layout(width='99.5%', height='780px'),)

def on_change_value_textarea_info(change):
    page = button_info.value
    if page == 'Промт':
        global promt_federallab
        promt_federallab = change['new']
    elif page == 'Чанки':
        pass
    elif page == 'Все чанки':
        pass

Textarea_info.observe(on_change_value_textarea_info, names='value')


In [167]:
# @title Вывод на экран оконного интерфеса
form_item_layout = Layout(display='flex', flex_flow='row', justify_content='space-between',  ) #border='1px solid black'
dialog_layout = Layout(display='flex', justify_content='flex-end', align_items = 'stretch' ) #, flex_flow='column' , flex-direction = 'column-reverse'

display.clear_output()
header.clear_output()
left.clear_output()
center.clear_output()

# with header:
#     display.display(image_flse)  button_load_all
with left:
    display.display(widgets.VBox([label_api_key,
                                  widgets.HBox([api_key, login_button], layout=form_item_layout),
                                  label_load_data,
                                  widgets.HBox([label_db_question, button_load_all], layout=form_item_layout),
                                  widgets.HBox([link_bd_question, button_load_db_question], layout=form_item_layout),
                                  label_db_question_exel,
                                  widgets.HBox([link_bd_question_exel, input_person_exel, button_load_db_question_exel], layout=form_item_layout),
                                  label_db_vector,
                                  widgets.HBox([link_bd_vector, button_load_db_vector], layout=form_item_layout),
                                  label_db_text,
                                  widgets.HBox([link_bd_text, button_load_db_text], layout=form_item_layout),
                                  label_promt,
                                  widgets.HBox([link_promt, button_load_promt], layout=form_item_layout),
                                  label_param_name,
                                  model_gpt,
                                  temperature_gpt,
                                  input_chank_threshold,
                                  input_top_documents,
                                #   widgets.HBox([input_top_documents, seed_gpt], layout=form_item_layout),
                                  input_question_norma,
                                  input_clear_dialogue_before_new_answer,
                                  widgets.HBox([input_question_dialog, input_question_dialog_window], layout=form_item_layout),
                                  widgets.HBox([input_verbose_dialog, button_dialog_new], layout=form_item_layout),
                                #   input_verbose_chank,
                                #   input_verbose_price,
                                  label_param_cache,
                                  input_cache_threshold,
                                  input_cache_number,
                                ]))
with center:
    display.display(widgets.VBox([output_dialogs,
                                  widgets.HBox([input_question_client,button_sent_question],layout=form_item_layout),
                                  input_qestion_from_cache,
                                ], layout = dialog_layout))
with right:
    display.display(button_info,
                    Textarea_info,
    )
# with footer:
#     print("This is output widget")